### CAMINHOS DOS ARQUIVOS:

In [14]:
url_cargas = '/content/CRUZ ALTA - CARGAS.xlsx' # Caminho do relatório de cargas do distrito que estamos validando.
url_visitas = '/content/CRUZ ALTA - VISITAS.xlsx' # Caminho do relatório de visitas do distrito que estamos validando.
url_log_de_tecnologia = '/content/CRUZ ALTA - LOG DE TECNOLOGIA.xlsx' # Caminho do relatório de log de tecnologia do distrito que estamos validando.
url_cargas_geral = r'C:\Users\coere\OneDrive\Área de Trabalho\Rafael\Projetos\Portifólio\GERAL - CARGAS.xlsx' # Caminho do relatório de cargas geral.
url_rotas = r'C:\Users\coere\OneDrive\Área de Trabalho\Rafael\Projetos\Portifólio\GERAL - ROTAS.xlsx' # Caminho do relatório de rotas geral.
data_inicial = '2025-04-16' # Data inicial que estamos validando.
data_final = '2025-04-21' # Data final que estamos validando.

### 01.ACUMULADO EM PDR VINCULADA:

In [5]:
# Importando a biblioteca pandas com o apelido pd.
import pandas as pd

# Lendo o arquivo Excel que importamos na variável url_visitas e armazenando na variável df_acumulado_em_pdr_vinculada.
df_acumulado_em_pdr_vinculada = pd.read_excel(url_visitas, header=1)

# Convertendo a coluna "DATA VISITA" para datetime.
df_acumulado_em_pdr_vinculada["DATA VISITA"] = pd.to_datetime(df_acumulado_em_pdr_vinculada["DATA VISITA"], dayfirst=True, errors="coerce")

# Criando as colunas OBESERVAÇÃO C.I e AJUSTE OPERAÇÃO.
df_acumulado_em_pdr_vinculada["OBSERVAÇÃO C.I."] = ""
df_acumulado_em_pdr_vinculada['AJUSTE OPERAÇÃO'] = ""

# Aplicando os filtros: 
# 1. ID contém "_".
# 2. C: VOLUME TOTAL RECEBIDO SEM PARTICIPANTE (Kg) > 0.
# 3. Coluna SITUAÇÃO = Aprovada.
# 4. Coluna TIPO VISITA = PRESENCIAL.
# 5. DATA VISITA entre data_inicial e data_final.
df_acumulado_em_pdr_vinculada_filtrado = df_acumulado_em_pdr_vinculada[
    (df_acumulado_em_pdr_vinculada["ID"].str.contains("_", na=False)) &
    (df_acumulado_em_pdr_vinculada["C: VOLUME TOTAL RECEBIDO SEM PARTICIPANTE (Kg)"] > 0) &
    (df_acumulado_em_pdr_vinculada['SITUAÇÃO'] == 'Aprovada') &
    (df_acumulado_em_pdr_vinculada['TIPO VISITA'] == 'PRESENCIAL') &
    (df_acumulado_em_pdr_vinculada["DATA VISITA"].between(data_inicial, data_final))
]

# Verificando se o DataFrame filtrado está vazio.
if df_acumulado_em_pdr_vinculada_filtrado.empty:
    print("Validação concluída: nenhum registro encontrado com os critérios definidos.")
else:
    # Filtro adicional: remover linhas com a frase específica na coluna 'OBSERVAÇÕES GERAIS DA VISITA'.
    df_acumulado_em_pdr_vinculada_filtrado = df_acumulado_em_pdr_vinculada_filtrado[
        ~df_acumulado_em_pdr_vinculada_filtrado["OBSERVAÇÕES GERAIS DA VISITA"].str.contains(
            "VALORES DE ACUMULADOS ATUALIZADOS ATRAVES DO PORTAL", case=False, na=False
        )
    ]

# Selecionando colunas desejadas para o relatório final que enviaremos para o cliente.
colunas_desejadas = [
    "ID", "DISTRITO FILIAL", "UF FILIAL", "CIDADE FILIAL", "RAZÃO SOCIAL FILIAL PDR", "CNPJ FILIAL PDR",
    "RESPONSÁVEL PDR", "LÍDER", "CONSULTOR", "DATA VISITA", "STATUS", "OBSERVAÇÃO C.I.",
    "AJUSTE OPERAÇÃO", "PDR FORNECEU DADOS DO ACUMULADO DA SAFRA?",
    "C: VOLUME TOTAL RECEBIDO SEM PARTICIPANTE (Kg)", "C: VOLUME BIOTECNOLOGIA SEM PATENTE VÁLIDA (Kg)",
    "C: VOLUME BIOTECNOLOGIA COM PATENTE VÁLIDA (Kg)", "C: VOLUME PARTICIPANTES (Kg)",
    "OBSERVAÇÕES GERAIS DA VISITA", "TIPO VISITA"
]

# Salvando o DataFrame em Excel.
df_acumulado_em_pdr_vinculada_filtrado.to_excel("ACUMULADO EM PDR VINCULADA.xlsx", columns = colunas_desejadas, index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/content/CRUZ ALTA - VISITAS.xlsx'

### ROTAS NÃO EXPORTADAS POR DISTRITO:

In [32]:
import pandas as pd
from datetime import datetime

# Lendo o arquivo Excel
df_rotas_nao_exportadas = pd.read_excel(url_rotas, header=4)

In [33]:
# Convertendo a coluna de data para datetime
df_rotas_nao_exportadas["DATA"] = pd.to_datetime(df_rotas_nao_exportadas["DATA"], dayfirst=True, errors="coerce")

df_rotas_nao_exportadas['DISTRITO FILIAL 2'] = ""

In [34]:
# Padroniza os nomes dos distritos
df_rotas_nao_exportadas['DISTRITO FILIAL'] = df_rotas_nao_exportadas['DISTRITO FILIAL'].str.strip()
# Padroniza os nomes dos distritos
df_rotas_nao_exportadas['DISTRITO FILIAL 2'] = df_rotas_nao_exportadas['DISTRITO FILIAL 2'].str.strip()

# Filtra apenas linhas com 'ID Visita Recorrente' vazio
df_filtrado = df_rotas_nao_exportadas[df_rotas_nao_exportadas['ID Visita Recorrente'].isnull()]

# Agrupa por DISTRITO FILIAL e pega a menor DATA (mais antiga)
datas_mais_antigas = (
    df_filtrado
    .groupby('DISTRITO FILIAL', as_index=False)['DATA']
    .min()
    .rename(columns={'DATA': 'NÃO_EXPORTADA'})
    .sort_values(by='NÃO_EXPORTADA')
)

In [35]:
# Agora, junta os distritos da sua lista (DISTRITO FILIAL 2)
# com as datas mais antigas, e para quem não tiver data, coloca a data de hoje

# Lista de distritos
distritos = [
    "PALOTINA-SOJA", "SANTA ROSA-SOJA", "PATO BRANCO-SOJA",
    "LAGOA VERMELHA-SOJA", "CASCAVEL-SOJA", "IJUI-SOJA",
    "CHAPECO-SOJA", "MARINGA-SOJA", "RIO DO SUL-SOJA",
    "LONDRINA-SOJA", "JATAI-SOJA", "GUARAPUAVA-SOJA",
    "CANOINHAS-SOJA", "CORNELIO PROCOP-SOJA", "PONTA GROSSA-SOJA",
    "JULIO DE CASTILHOS-SOJA", "SARANDI-SOJA", "CARAZINHO-SOJA",
    "IBIRUBA-SOJA", "SANTO ANGELO-SOJA", "CRUZ ALTA-SOJA",
    "PORTO ALEGRE-SOJA", "SAO GABRIEL-SOJA", "PASSO FUNDO-SOJA"
]

# Cria DataFrame com todos distritos
df_rotas_nao_exportadas2 = pd.DataFrame({'DISTRITO FILIAL 2': distritos})

# Faz o merge (união) com datas_mais_antigas
df_rotas_nao_exportadas3 = df_rotas_nao_exportadas2.merge(
    datas_mais_antigas,
    left_on='DISTRITO FILIAL 2',
    right_on='DISTRITO FILIAL',
    how='left'
)

# Preenche datas faltantes com data de hoje
df_rotas_nao_exportadas3['NÃO_EXPORTADA'] = df_rotas_nao_exportadas3['NÃO_EXPORTADA'].fillna(datetime.today().strftime('%Y-%m-%d'))

# Se quiser, elimina a coluna 'DISTRITO FILIAL' que veio do merge
df_rotas_nao_exportadas3 = df_rotas_nao_exportadas3.drop(columns=['DISTRITO FILIAL'])

# Resultado final
df_rotas_nao_exportadas3

,DISTRITO FILIAL 2,NÃO_EXPORTADA
0,PALOTINA-SOJA,2025-04-27
1,SANTA ROSA-SOJA,2025-04-27
2,PATO BRANCO-SOJA,2025-04-28
3,LAGOA VERMELHA-SOJA,2025-04-18
4,CASCAVEL-SOJA,2025-04-27
5,IJUI-SOJA,2025-04-23
6,CHAPECO-SOJA,2025-04-24
7,MARINGA-SOJA,2025-04-27
8,RIO DO SUL-SOJA,2025-04-27
9,LONDRINA-SOJA,2025-04-27


In [36]:
# Ordena pelo campo NÃO_EXPORTADA
df_rotas_nao_exportadas_final = df_rotas_nao_exportadas3.sort_values(by='NÃO_EXPORTADA')

In [37]:
df_rotas_nao_exportadas_final

,DISTRITO FILIAL 2,NÃO_EXPORTADA
18,IBIRUBA-SOJA,2025-03-27
23,PASSO FUNDO-SOJA,2025-04-09
14,PONTA GROSSA-SOJA,2025-04-11
21,PORTO ALEGRE-SOJA,2025-04-17
3,LAGOA VERMELHA-SOJA,2025-04-18
11,GUARAPUAVA-SOJA,2025-04-21
17,CARAZINHO-SOJA,2025-04-23
5,IJUI-SOJA,2025-04-23
6,CHAPECO-SOJA,2025-04-24
12,CANOINHAS-SOJA,2025-04-25


In [38]:
# Salva em Excel
df_rotas_nao_exportadas3.to_excel("ROTAS NÃO EXPORTADAS.xlsx", index=False)

### ÚLTIMA VALIDAÇÃO:

In [42]:
import pandas as pd

datas_personalizadas = {
    "CHAPECO-SOJA": "2025-04-23",
    "PONTA GROSSA-SOJA": "2025-04-10",
    "GUARAPUAVA-SOJA": "2025-04-20",
    "JULIO DE CASTILHOS-SOJA": "2025-04-24",
    "IBIRUBA-SOJA": "2025-04-24",
    "RIO DO SUL-SOJA": "2025-04-26",
    "SÃO GABRIEL-SOJA": "2025-04-24",
    "CRUZ ALTA-SOJA": "2025-04-26",
    "SARANDI-SOJA": "2025-04-19",
    "CORNELIO PROC-SOJA": "2025-04-19",
    "PORTO ALEGRE-SOJA": "2025-04-16",
    "LAGOA VERMELHA-SOJA": "2025-04-17",
    "SANTO ANGELO-SOJA": "2025-04-25",
    "SANTA ROSA-SOJA": "2025-04-24",
    "CANOINHAS-SOJA": "2025-04-24",
    "CARAZINHO-SOJA": "2025-04-22",
    "IJUI-SOJA": "2025-04-22",
    "PATO BRANCO-SOJA": "2025-04-21",
    "CASCAVEL-SOJA": "2025-03-13",
    "JATAI-SOJA": "2025-03-30",
    "LONDRINA-SOJA": "2025-04-05",
    "MARINGA-SOJA": "2025-04-05",
    "PALOTINA-SOJA": "2025-03-02"
}

df_distritos = pd.DataFrame([
    {
        'DISTRITO FILIAL': distrito,
        'ATÉ': pd.to_datetime(data)
    }
    for distrito, data in datas_personalizadas.items()
])

In [43]:
df_distritos

,DISTRITO FILIAL,ATÉ
0,CHAPECO-SOJA,2025-04-23
1,PONTA GROSSA-SOJA,2025-04-10
2,GUARAPUAVA-SOJA,2025-04-20
3,JULIO DE CASTILHOS-SOJA,2025-04-24
4,IBIRUBA-SOJA,2025-04-24
5,RIO DO SUL-SOJA,2025-04-26
6,SÃO GABRIEL-SOJA,2025-04-24
7,CRUZ ALTA-SOJA,2025-04-26
8,SARANDI-SOJA,2025-04-19
9,CORNELIO PROC-SOJA,2025-04-19


In [44]:
# Sort the DataFrame by the 'ATÉ' column in ascending order (oldest date first)
df_distritos_sorted = df_distritos.sort_values(by='ATÉ')

# Save the sorted DataFrame to an Excel file
df_distritos_sorted.to_excel("df_distritos_ordenado.xlsx", index=False)

### CARGAS A VALIDAR

In [20]:
import pandas as pd
import openpyxl
from datetime import datetime, timedelta

# Criando o DataFrame através do relatório em excel.
df_total_de_cargas = pd.read_excel(url_cargas_geral, header=1)

In [45]:
# Convertendo a coluna de data para datetime.
df_total_de_cargas["DATA"] = pd.to_datetime(df_total_de_cargas["DATA"], dayfirst=True, errors="coerce")

In [46]:
# Padroniza os nomes dos distritos.
df_total_de_cargas['DISTRITO FILIAL'] = df_total_de_cargas['DISTRITO FILIAL'].str.strip()

In [47]:
# prompt: USAR O df_total_de_cargas FILTRANDO PELA COLUNA DISTRITO FILIAL E CONTANDO A QUANTIDADE DE LINHAS

# Group by 'DISTRITO FILIAL' and count the number of rows in each group
distrito_counts = df_total_de_cargas.groupby('DISTRITO FILIAL').size().reset_index(name='COUNT')

In [48]:
# Junta os dois dataframes por 'DISTRITO FILIAL'
df_datas_unificadas = pd.merge(
    df_distritos,
    df_rotas_nao_exportadas3,
    left_on='DISTRITO FILIAL',   # coluna do primeiro dataframe
    right_on='DISTRITO FILIAL 2',  # coluna do segundo dataframe
    how='inner'  # apenas os que estão nos dois
)

In [49]:
# Criando a coluna "CARGAS P VALIDAR" no df_datas_unificadas
df_datas_unificadas['CARGAS P VALIDAR'] = ''

In [50]:
# # Iterate through each row in df_datas_unificadas
for index, row in df_datas_unificadas.iterrows():
    distrito = row['DISTRITO FILIAL']
    data_inicial = row['ATÉ'] + pd.Timedelta(days=1)
    data_final = row['NÃO_EXPORTADA'] - pd.Timedelta(days=1)

    # Filter df_total_de_cargas based on distrito, data_inicial, and data_final
    cargas_filtradas = df_total_de_cargas[
        (df_total_de_cargas['DISTRITO FILIAL'] == distrito) &
        (df_total_de_cargas['DATA'] >= data_inicial) &
        (df_total_de_cargas['DATA'] <= data_final)
    ]

    # Get the number of rows in the filtered DataFrame
    quantidade_linhas = len(cargas_filtradas)

    # Update the 'CARGAS P VALIDAR' column in df_datas_unificadas
    df_datas_unificadas.loc[index, 'CARGAS P VALIDAR'] = quantidade_linhas

In [51]:
# Remove rows where 'CARGAS P VALIDAR' is zero
df_datas_unificadas = df_datas_unificadas[df_datas_unificadas['CARGAS P VALIDAR'] != 0]

In [52]:
# Order the 'CARGAS P VALIDAR' column in descending order
df_datas_unificadas = df_datas_unificadas.sort_values(by='CARGAS P VALIDAR', ascending=False)
df_datas_unificadas

,DISTRITO FILIAL,ATÉ,DISTRITO FILIAL 2,NÃO_EXPORTADA,CARGAS P VALIDAR
15,PATO BRANCO-SOJA,2025-04-21,PATO BRANCO-SOJA,2025-04-28,33
20,PALOTINA-SOJA,2025-03-02,PALOTINA-SOJA,2025-04-27,4


In [53]:
# Assuming df_datas_unificadas2 is defined in the previous code cell
df_datas_unificadas.to_excel("ROTAS.xlsx", index=False)